In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from timer import Timer
import pickle

In [5]:
maxFloat16 = np.finfo(np.float16).max
neuronMax = 100
neuronMin = -100
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return NeuralNetwork.sigmoidDerivative(x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def sigmoidDerivative(x):
        return x * (1 - x)

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return NeuralNetwork.reluDerivative(x)
        return np.maximum(0, x)

    @staticmethod
    def reluDerivative(x):
        return 1.0 * (x > 0)

    @staticmethod
    def softmax(x):
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps, axis=1, keepdims=True)

    @staticmethod
    def softmaxDerivative(x):
        return x * (1 - x)

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )

    def backpropagation(self, target, learning_rate):
        output_error = target - self.layers[-1]

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error, self.weights[i].T) * derivative

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.dot(self.layers[i].T, output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += learning_rate * weight_adjustments
            self.biases[i] += learning_rate * bias_adjustments

            # Set the output error for the next iteration
            output_error = layer_error


In [9]:
x = NeuralNetwork([30_000, 20, 20, 3], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
nnTimer = Timer(precision=3)


for i in range(1):
    nnTimer.tic()
    x.loadInput(np.random.rand(30_000))
    x.forward()
    nnTimer.toc()
    print(x.readOutput())
nnTimer.stats()

[1.23644208e+08 1.19282296e+08 0.00000000e+00]
Total time: 0 s
Total calls: 1
Average time: 0 ms


In [6]:
sth = np.random.rand(3)

In [10]:
x.backpropagation(sth, 0.1)
x.forward()
print(x.readOutput())

ValueError: operands could not be broadcast together with shapes (20,) (3,) 

In [59]:
np.finfo(np.float32).max

3.4028235e+38

In [62]:
pickle.dump(x, open("nn.pkl", "wb"))

In [13]:
np.matmul(
    np.random.randint(size=10, low=-10, high=10),
    np.random.randint(size=10, low=-10, high=10),
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4106462344.py, line 1)